In [1]:
import pandas as pd
import numpy as np

In [2]:
businesses = pd.read_csv('../data/businesses.csv')
inspections = pd.read_csv('../data/inspections.csv')
sanitation = pd.read_csv('../data/sanitation.csv')
crime = pd.read_csv('../data/crimes.csv')
crime['date'] = pd.to_datetime(crime['date']).apply(lambda d : d.date())
garbage = pd.read_csv('../data/garbage.csv')

/Library/Python/2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (6,7,24,38) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [3]:
def find_business_by_license(license):
    
    filtered = businesses[businesses['license_id'] == license]
    
    return filtered
    

In [4]:
inspection_subset = inspections.loc[:, ['inspection_id', 'inspection_date', 
                                  'facility_type', 'inspection_type',
                                  'latitude', 'longitude', "license_", "results"]]
inspection_subset.shape

(135092, 8)

In [5]:
find_business_by_license(1991029)

,Unnamed: 0,account_number,address,application_created_date,application_requirements_complete,application_type,business_activity,business_activity_id,city,conditional_approval,...,longitude,payment_date,police_district,precinct,site_number,ssa,state,ward,ward_precinct,zip_code
611423,611423,346577,2715 N MILWAUKEE AVE,2009-08-19T00:00:00.000,2009-08-20T00:00:00.000,ISSUE,Retail Sales of Perishable Foods,775,CHICAGO,N,...,-87.709689,2009-08-20T00:00:00.000,14,17,1,NaN,IL,35,35-17,60647


In [6]:
valid_result_inspections = pd.DataFrame(inspection_subset[~(~inspection_subset['results'].str.contains("Pass")) \
                                             & (~inspection_subset['results'].str.contains('Fail'))])

In [7]:
valid_result_inspections['inspection_date'] = pd.to_datetime(inspection_subset['inspection_date']).apply(lambda d : d.date())
valid_result_inspections.shape

(92406, 8)

In [9]:
inspections_with_biz_info = pd.merge(valid_result_inspections, businesses, left_on='license_', right_on='license_id', how='left')

In [10]:
inspections_with_biz_info.head()

,inspection_id,inspection_date,facility_type,inspection_type,latitude_x,longitude_x,license_,results,Unnamed: 0,account_number,...,longitude_y,payment_date,police_district,precinct,site_number,ssa,state,ward,ward_precinct,zip_code
0,1184245,2012-04-06,Restaurant,Canvass,41.886294,-87.642644,1842843,Pass,248061,320408,...,-87.642644,2007-08-17T00:00:00.000,12,18,1,NaN,IL,42,42-18,60661
1,478757,2010-12-09,Restaurant,Complaint,41.930605,-87.709689,1991029,Pass w/ Conditions,611423,346577,...,-87.709689,2009-08-20T00:00:00.000,14,17,1,NaN,IL,35,35-17,60647
2,1115054,2013-10-03,Restaurant,Canvass Re-Inspection,41.875314,-87.627473,1904806,Pass,762614,331323,...,-87.627473,2008-05-09T00:00:00.000,1,49,1,NaN,IL,4,4-49,60605
3,1575282,2015-09-09,Children's Services Facility,License,41.970113,-87.713443,2215517,Pass,194697,27889,...,-87.713443,NaN,17,22,1,60,IL,33,33-22,60625
4,417230,2010-10-14,Restaurant,Canvass,41.802134,-87.621203,1954887,Pass,491638,340956,...,-87.621203,2009-02-25T00:00:00.000,2,40,1,52,IL,3,3-40,60615


In [27]:
d = crime['date'].value_counts()
d

2003-08-21    1516
2003-10-16    1455
2003-08-29    1439
2003-09-23    1438
2003-10-17    1437
2003-09-12    1401
2003-10-09    1379
2003-07-19    1379
2003-09-06    1376
2003-08-05    1373
2003-07-28    1369
2003-08-08    1364
2003-08-15    1358
2003-08-01    1350
2003-10-08    1348
2003-09-11    1344
2003-08-13    1343
2003-09-09    1332
2003-08-25    1330
2003-08-14    1325
2003-08-06    1323
2003-07-25    1323
2003-08-20    1323
2003-08-16    1323
2003-09-15    1322
2003-07-29    1318
2003-09-13    1312
2003-08-07    1311
2003-09-19    1306
2003-07-26    1306
              ... 
2013-12-29       1
2008-02-21       1
2004-08-25       1
2007-07-27       1
2006-03-02       1
2004-07-27       1
2009-02-28       1
2012-04-03       1
2003-11-19       1
2007-04-16       1
2009-01-28       1
2007-02-01       1
2004-10-30       1
2009-06-16       1
2012-11-29       1
2007-11-18       1
2011-06-29       1
2006-03-25       1
2012-09-22       1
2014-01-07       1
2007-03-03       1
2011-03-14  

In [28]:
def get_nearby_crimes(inspection_row):
    lat = inspection_row['latitude_x']
    lon = inspection_row['longitude_x']
    
    lat_range = (crime['latitude'] - lat < 0.0003)
    lon_range = (crime['longitude'] - lon < 0.0003)
    date_match = crime['date'] == inspection_row['inspection_date']
    return len(crime[date_match & lat_range & lon_range])


In [ ]:
crime_count = inspections_with_biz_info.apply(get_nearby_crimes, axis=1)